# QC on WaferGen PCR SmartChip


## Get Data

In [1]:
import pandas, os, glob
import logging
import matplotlib.pyplot as plt
import seaborn 
%matplotlib inline
FORMAT = "%(asctime)s:%(levelname)s:%(message)s"
logging.basicConfig(format=FORMAT)
LOG= logging.getLogger()
LOG.setLevel(logging.DEBUG)

class FilePaths():
    def __init__(self):
        self.dire =  r'/home/b3053674/Documents/WaferGen/2D_TGFb'
        self.ncl = os.path.join(self.dire, 'GSS2242_NCGeneSet')
        self.ecm = os.path.join(self.dire, 'GSS2266_ECMGeneSet')
        self.ncl_raw_data_files = sorted(glob.glob(os.path.join(self.ncl, '*WellData*')  )  )
        self.ecm_raw_data_files = sorted(glob.glob(os.path.join(self.ecm, '*WellData*')  )  )
        self.QC = os.path.join(self.dire, 'QCResults')
        
        self._make_qc()
        
    def _make_qc(self):
        """
        
        """
        if os.path.isdir(self.QC)!=True:
            os.makedirs(self.QC)


F=FilePaths()

def parse_data(data1, data2, labels1, labels2):
    """
    Parse smart-chip PCR data
    
    args:
        data1: data from chip 1 (NCL or ECM)
    
        data2 Data from chip2 (NCL or ECM)
        
        labels1: labels for chips (NCL or ECM)
        
        labels2: Condition labels (Neonatal, IR and Adult)
    
    """

    ## construct python dict for data
    dct = {}
    for i in range(len(labels1)):
        dct[labels1[i]] = {}

    for i in range(len(labels1)):
        for j in range(len(labels2)):
            if labels1[i]=='Chip1NCL':
                dct[labels1[i]][labels2[j]] = pandas.read_csv(data1[j], sep='\t')
            elif labels1[i]=='Chip2ECM':
                dct[labels1[i]][labels2[j]] = pandas.read_csv(data2[j], sep='\t')

    ## remove unwanted columns
    columns = ['Assay', 'Sample','Ct']
    dct2 = {}
    for chip in dct:
        dct2[chip] = {}
        for treatment in dct[chip]:
            dct2[chip][treatment] = dct[chip][treatment][columns]
            
    ## parse data into one pandas dataframe

    df_dct = {}
    for chip in dct2:
        df_dct[chip] = pandas.concat(dct2[chip])
        
    return pandas.concat(df_dct)

    
ncl_raw_data = sorted(glob.glob(os.path.join(F.ncl, '*WellData*')  )  )
ecm_raw_data = sorted(glob.glob(os.path.join(F.ecm, '*WellData*')  )  )

labels1 = ['Chip1NCL', 'Chip2ECM']
labels2 = ['Neonatal', 'IR', 'Adult']
    
    
LOG.info('Parsing data from file into python ')
df = parse_data(F.ncl_raw_data_files, F.ecm_raw_data_files, labels1, labels2)

def format_data(df):
    """
    Make data look pretty
    """
    LOG.debug('DataFrame now looks like this:\n{}'.format(df.head()) )
    df=df.reset_index()
    df=df.drop('level_2', axis=1)
    df.columns = ['Chip', 'CellType', 'Gene', 'Sample', 'Ct']
    # df4=df3.set_index(['Chip','CellType','Gene','Sample'])
    df = df.dropna()
    return df

    
df = format_data(df)

2017-07-12 17:27:17,677:INFO:Parsing data from file into python 
2017-07-12 17:27:17,781:DEBUG:DataFrame now looks like this:
                   Assay     Sample        Ct
Chip1NCL Adult 0    ABL1  Tgfb_48_4  23.98787
               1    ACTB  Tgfb_48_4  21.11110
               2   CDC42  Tgfb_48_4  24.20522
               3  CDKN2A  Tgfb_48_4  28.10812
               4   NFKB2  Tgfb_48_4  28.18900



## Plot some graphs that serve as QC

In [ ]:
seaborn.set_context(context='poster', font_scale=3)

SAVE = True

PLOT_CYCLE_THRESHOLD_DISTRIBUTIONS = False
PLOT_DENSITY = True




if PLOT_CYCLE_THRESHOLD_DISTRIBUTIONS:
    
# seaborn.boxplot(data=df3, x='Chip', y='Ct', hue='Chip')
    plt.figure()
    seaborn.boxplot(data=df, x='Chip', y='Ct', hue='CellType')
    plt.title('Cycle Threshold Distributions')
    plt.legend(loc=(1,0.5))
    if SAVE:
        plt.savefig(os.path.join(F.QC, 'RawCtDistributionsPerChipPerCellType.jpeg'), bbox_inches='tight', dpi=300)
        
if PLOT_DENSITY:
    plt.figure()
    for label, d in df.groupby(by='Chip'):
        seaborn.distplot(d['Ct'], label=label)
        plt.legend(loc=(1,0.5))
        plt.title('Comparison of Ct Distributions Accross \nTwo PCR Chips')
        plt.ylabel('Normalized Frequency')
        if SAVE:
            plt.savefig(os.path.join(F.QC, 'DensityPlotCtDistributions.jpeg'), bbox_inches='tight', dpi=300)


# PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn import preprocessing


PLOT_PCA = False
PLOT_EXPLAINED_VARIANC_RATIO = False
SAVE=False

df_indexed = df.set_index(['Chip', 'CellType', 'Gene', 'Sample'])


df_indexed=df_indexed.unstack()

df_indexed.columns = df_indexed.columns.droplevel(0)

##imputation with median prior to feeding into PCA
imp =preprocessing.Imputer(strategy='median', axis=1)
imp.fit(df_indexed)
df_imputed = (imp.transform(df_indexed))
df_imputed = pandas.DataFrame(df_imputed, columns=df_indexed.columns, index =df_indexed.index ) 
df_imputed = df_imputed[sorted(df_imputed.keys())]


print(df_indexed)

# print(df_imputed.head())


if PLOT_EXPLAINED_VARIANC_RATIO :
    pca = PCA(n_components=10)
    pca = pca.fit(df_imputed)
    plt.figure()
    plt.title('Explained Variance Ratio for First \n 10 Principle Components')
    seaborn.pointplot(x=range(len(pca.explained_variance_ratio_)), y = pca.explained_variance_ratio_)
    plt.ylabel('Explained Variance Ratio')
    plt.xlabel('Principle Component')
    if SAVE:
        plt.savefig(os.path.join(F.QC, 'ExplainedVarianceRatioPlot.jpeg'))
        
if PLOT_PCA:
    pca = PCA(n_components=2)
    pca = pca.fit(df_imputed)
    print(pandas.DataFrame(pca.transform(df_imputed), index=df_imputed.index) )
    
    
    # pca.fit(df_indexed)/



In [ ]:
##PCA Python example

print(__doc__)

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

iris = datasets.load_iris()

print (iris)

X = iris.data
y = iris.target
target_names = iris.target_names

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

print (X_r)

lda = LinearDiscriminantAnalysis(n_components=2)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
# plt.legend(loc='best', shadow=False, scatterpoints=1)
# plt.title('PCA of IRIS dataset')

# plt.figure()
# for color, i, target_name in zip(colors, [0, 1, 2], target_names):
#     plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
#                 label=target_name)
# plt.legend(loc='best', shadow=False, scatterpoints=1)
# plt.title('LDA of IRIS dataset')

# plt.show()

